# Kaggle link: https://www.kaggle.com/code/l1ghtsource/yappy-faces

In [1]:
! pip3 install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 15.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.6 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-19.24.4-cp310-cp310-linux_x86_64.whl size=3327210 sha256=54cc015d9861865d52177ff750d6b4da82edccbfbcf6788ea6547281fc381070
  Stored in directory: /root/.cache/pip/wheels/08/5e/55/a7db8d57920eb2c2aa41c36dc4c6d3cd12323865de191a7211
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566165 sha256=80ef57217dcce2d2edcd20f04499a7cb467d8895f18752a0c4dc26ee7e4d02bc
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built dlib face-recognition-models


In [2]:
! pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


In [3]:
! pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 14.8 MB/s eta 0:00:00 0:00:01


In [4]:
! pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 54.2 MB/s eta 0:00:00:00:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110720 sha256=e38512d99f525a202c9dd9b98784858417a4ecf9110c27a7c31925c29c8c3c23
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [5]:
import pandas as pd

df = pd.read_csv('/kaggle/input/yaaaaaaaaaaaaaaaaaaaaaaaaaaapy/yappy_hackaton_2024_400k.csv')
video_urls = df.link.tolist()[4000:6000]

In [6]:
import os
import requests
import cv2
import face_recognition
import numpy as np
import pandas as pd
import hdbscan
from collections import defaultdict
import moviepy.editor as mp
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [7]:
def download_video(url, output_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f'Failed to download video from {url}. Status code: {response.status_code}')
    except Exception as e:
        print(f'Error downloading video {url}: {e}')

        
def extract_frames(video_path, frame_interval=8):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frames = []
    count = 0
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    while success:
        if count % (frame_interval * fps) == 0:
            frames.append(image)
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    return frames


def extract_face_embeddings(frames):
    face_embeddings = defaultdict(int)
    for frame in frames:
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        for encoding in face_encodings:
            encoding_tuple = tuple(encoding)
            face_embeddings[encoding_tuple] += 1
    popular_embeddings = [np.array(encoding) for encoding, count in face_embeddings.items() if count > 0] # there is no need
    print(f'Find {len(popular_embeddings)} faces')
    return popular_embeddings


def process_video(url, temp_dir='temp_videos', frame_interval=5):
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    video_path = os.path.join(temp_dir, os.path.basename(url))
    download_video(url, video_path)
    frames = extract_frames(video_path, frame_interval)
    embeddings = extract_face_embeddings(frames)
    os.remove(video_path)
    return embeddings


def save_embeddings_to_dataframe(url, embeddings):
    data = []
    for emb in embeddings:
        data.append([url, emb])
    df = pd.DataFrame(data, columns=['link', 'emb'])
    return df


def cluster_embeddings(embedding_data):
    embeddings = np.array(embedding_data['emb'].tolist())
    
    if len(embeddings) == 0:
        return embedding_data
    
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    
    pca = PCA(n_components=50)
    embeddings_reduced = pca.fit_transform(embeddings_scaled)
    
    print('Clustering')
    clusterer = hdbscan.HDBSCAN(min_cluster_size=10, core_dist_n_jobs=-1)
    
    cluster_labels = clusterer.fit_predict(embeddings_reduced)
    embedding_data['cluster'] = cluster_labels
    
    return embedding_data


def create_cluster_dict(clustered_data):
    cluster_dict = defaultdict(list)
    for _, row in clustered_data.iterrows():
        cluster_dict[row['cluster']].append(row['link'])
    return dict(cluster_dict)


def main(video_urls):
    all_embeddings = pd.DataFrame(columns=['link', 'emb'])
    global emb_df
    
    for url in tqdm(video_urls):
        try:
            embeddings = process_video(url)
            if embeddings:
                video_df = save_embeddings_to_dataframe(url, embeddings)
                all_embeddings = pd.concat([all_embeddings, video_df], ignore_index=True)
                emb_df = all_embeddings.copy()
        except Exception as e:
            print(f'Error processing video {url}: {e}')
    
    if len(all_embeddings) > 0:
        clustered_data = cluster_embeddings(all_embeddings)
        cluster_dict = create_cluster_dict(clustered_data)
        return cluster_dict
    else:
        return {}

In [8]:
clusters = main(video_urls)
len(clusters)

  0%|          | 1/2000 [00:05<2:54:34,  5.24s/it]

Find 2 faces


  0%|          | 2/2000 [00:09<2:34:18,  4.63s/it]

Find 1 faces


  0%|          | 3/2000 [00:18<3:34:46,  6.45s/it]

...

Find 11 faces


100%|██████████| 2000/2000 [5:12:30<00:00,  9.38s/it]

Find 0 faces


Clustering


45

In [14]:
for i in range(len(clusters) - 1):
    clusters[i] = list(set(clusters[i]))

In [24]:
good_clusters = {}
for i in range(len(clusters) - 1):
    if len(clusters[i]) > 3 and i != 43: # 43 just random girls idk why
        good_clusters[i] = clusters[i]

In [26]:
df_clustered = pd.DataFrame(pd.Series(good_clusters))

In [27]:
df_clustered.to_csv('face_clusters_on_2000_videos.csv')

In [29]:
good_clusters

{0: ['https://cdn-st.rutubelist.ru/media/84/6b/aec59c63443e847c90bd8a33efc8/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/57/e9/412b21154ed0b315c7007d592f6f/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/57/82/b1389a854516ba0209c2ea6d7ef4/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/07/1c/253740ff452b88183ce0de3d64f3/fhd.mp4'],
 2: ['https://cdn-st.rutubelist.ru/media/3c/f1/1dcd53e045aaa842944a08caff7c/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/91/17/37a8ee614008a09865e6ee33eca2/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/a0/13/cd836fef460c8adbf8e92a25b008/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/44/5b/8edbb1b4474794c478b82c378028/fhd.mp4'],
 3: ['https://cdn-st.rutubelist.ru/media/65/5a/22f3e2c549efa984a4bec32f8802/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/d7/f8/c6807da84b889cecd4757cce7f95/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/d0/79/aee1b4754dd8910d141bf5f587cd/fhd.mp4',
  'https://cdn-st.rutubelist.ru/media/8d/f0/eaa8c0304060adbc7d1d175e73ee/fhd